In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', 100)

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import re
import string
import nltk
import xgboost
import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV

import sys
if not 'Notebooks/Individual/Jake' in sys.path:
    sys.path.append('Notebooks/Individual/jake')
from functions import ScoreModel#, CleanText

import IPython
sound_file = '../../../data/sounds/puzzle_solved_jingle.wav'

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, plot_confusion_matrix

from textblob import TextBlob, Word
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ultim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ultim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("../../../data/thanos.csv")
df.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable', 'Building', 'Comedy', 'Co-op', 'Mod', 'First-Per...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable', 'Building', 'Comedy', 'Co-op', 'Mod', 'First-Per...",I completely suck at making anything on here and death run keeps ending in countless deaths beca...,True,True
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1990's"", 'Shooter', 'Multiplayer', 'First-Person', ...",There is not much to say about this old game that hasn't been said really. Many people have play...,True,True


In [3]:
df["voted_up"].value_counts(normalize=True)

True     0.505888
False    0.494112
Name: voted_up, dtype: float64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23183 entries, 0 to 23182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    23183 non-null  int64 
 1   appid      23183 non-null  int64 
 2   app_title  23183 non-null  object
 3   app_tags   23183 non-null  object
 4   review     23183 non-null  object
 5   fps        23183 non-null  bool  
 6   voted_up   23183 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 951.0+ KB


In [7]:
def CleanText(reviews):
    stopwords = nltk.corpus.stopwords.words('english')
    punct = string.punctuation
    lemma = nltk.WordNetLemmatizer()
    
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    
    return reviews

In [5]:
df['clean_review'] = df['review'].apply(lambda x: CleanText(x.lower()))

df.head()

NameError: name 'CleanText' is not defined

In [8]:
vector = TfidfVectorizer(analyzer=CleanText, ngram_range=(2, 2))
X = vector.fit_transform(df["review"])

X_df = pd.DataFrame(X.toarray())
X_df.columns = vector.get_feature_names()
X_df.head(3)

,,0,00,000,0000000000,00000000000,000000000000,0000000000000000,0000000000000000000,00000000000000000000,...,𝚝𝚑𝚎,𝚝𝚑𝚒𝚜,𝚝𝚑𝚛𝚎𝚎𝚙𝚊𝚛𝚝,𝚝𝚘,𝚠𝚊𝚗𝚗𝚊,𝚠𝚊𝚗𝚝,𝚠𝚑𝚊𝚝,𝚠𝚘𝚛𝚕𝚍,𝚢𝚘𝚞𝚛,𝟣
0,0.43771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y = df["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=57)

first_half_model = RandomForestClassifier(n_estimators=100, random_state=57, n_jobs=-1)
first_half_model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=57)

In [9]:
ScoreModel(first_half_model, X_train, y_train)

Accuracy:   0.9985621441306723
F1 Score:   0.9985822038223785
Recall:     0.9993189557321226
Precision:  0.9978465374589142
ROC_AUC:    0.9985518644814279


In [10]:
ScoreModel(first_half_model, X_test, y_test)

Accuracy:   0.7877846790890269
F1 Score:   0.7824548991864168
Recall:     0.7580534612748457
Precision:  0.8084795321637427
ROC_AUC:    0.7879912893587572


In [11]:
sums = X.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(X_df.columns): 
    data1.append( (term, sums[0, col] )) 
ranking = pd.DataFrame(data1, columns = ['term', 'rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
print ("\n\nWords : \n", words.head(10))



Words : 
        term         rank
24724  game  1474.627629
7613      I   911.370448
33224  play   518.401657
0             489.557583
25337  good   484.404476
24551   fun   475.089970
29171  like   373.178621
25104   get   352.586144
13537   The   343.876675
13575  This   338.590873


RF_grid = RandomForestClassifier(random_state=57)

param_grid = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
}

grid_RF = GridSearchCV(RF_grid, param_grid, n_jobs=-1, scoring="precision", cv=2, verbose=3)
grid_RF.fit(X_train, y_train)

print("Training Scores")
print(ScoreModel(grid_RF, X_train, y_train))

print("Test Scores")
print(ScoreModel(grid_RF, X_test, y_test))

In [ ]:
IPython.display.Audio(sound_file, autoplay=True, rate=1000)